### 새로운 모델

- 독립변수로 AMT를 예측하는 모델링 > CNT와 CSTMR_CNT를 예측 후 두 값으로 AMT를 예측하는 수정 예측 모델
    - 독립변수가 이용객 수와 오히려 상관관계가 있을 것으로 판단 (우선 corr 뽑아보기)
- Label Encoder > dummies 후 One-hot 인코딩으로 진행하는 인코딩/디코딩
- 카드 가맹 시도와 거주지 시도 비교하여 새로 만드는 현지인 여부 확인 컬럼 : 모델이 더 세분화하여 파악할 수 있도록??
- 외부 데이터 LSTM 시계열 분석을 통해 202004, 202007 예측 후 독립변수로 학습

**Libraries**

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [17]:
from matplotlib import font_manager, rc
# window >
font_name = font_manager.FontProperties(fname="/Windows/Fonts/NanumGothicCoding.ttf").get_name()
# linux >
# font_name = font_manager.FontProperties(fname="/home/sundooedu/.local/share/fonts/NanumGothicCoding.ttf").get_name()
# gram >
# font_name = font_manager.FontProperties(fname="/Windows/Fonts/gulim.ttc").get_name()
rc('font', family=font_name)

# Basic

In [10]:
import os

os.getcwd()

'/content'

In [ ]:
jeju = pd.read_csv('./Data/jeju_data_ver1/201901-202003.csv')

In [ ]:
# jeju.describe(), jeju.info()
jeju.isnull().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      87213
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      147787
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
dtype: int64

결측값이 약 20만개정도 있는 것을 확인할 수 있다.

In [ ]:
jeju[jeju['HOM_CCG_NM'].isnull()]

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
420,201901,강원,강릉시,과실 및 채소 소매업,세종,NaN,30s,1,2,3,58000,3
421,201901,강원,강릉시,과실 및 채소 소매업,세종,NaN,40s,1,3,3,263000,4
1189,201901,강원,강릉시,기타 대형 종합 소매업,세종,NaN,20s,1,1,3,176540,3
1190,201901,강원,강릉시,기타 대형 종합 소매업,세종,NaN,20s,2,1,4,234880,6
1191,201901,강원,강릉시,기타 대형 종합 소매업,세종,NaN,30s,1,2,4,177640,5
...,...,...,...,...,...,...,...,...,...,...,...,...
24697496,202003,충북,충주시,한식 음식점업,세종,NaN,30s,2,2,5,188500,8
24697497,202003,충북,충주시,한식 음식점업,세종,NaN,40s,1,3,18,1056500,28
24697498,202003,충북,충주시,한식 음식점업,세종,NaN,50s,1,4,25,770150,35
24697499,202003,충북,충주시,한식 음식점업,세종,NaN,50s,2,4,4,19500,4


결측값은 가맹점 시군구와 카드 사용자의 거주 시군구이다.\
해당 값은 시도 값이 '세종'으로, 세종의 시군구가 NaN 값으로 들어간 것을 알 수 있다.\
이후 해당 컬럼을 독립변수에서 제외 할 예정이므로, 결측값을 따로 처리하지 않는다.

# Data Cleansing

In [ ]:
# 날짜 처리

def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

data = jeju.copy()
data = data.fillna('')

data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))

data = data.drop(['REG_YYMM'], axis = 1)

data

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,month,year
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,1,2019
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,1,2019
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,1,2019
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,1,2019
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4,3,2020
24697788,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3,3,2020
24697789,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6,3,2020
24697790,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6,3,2020


In [ ]:
# 데이터 정제

data = data.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis = 1)

columns =['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE',
          'SEX_CTGO_CD', 'FLC', 'year', 'month']
data = data.groupby(columns).sum().reset_index(drop = False)
data

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3
...,...,...,...,...,...,...,...,...,...,...,...
1057389,충북,휴양콘도 운영업,충북,70s,2,5,2019,3,3,148000,4
1057390,충북,휴양콘도 운영업,충북,70s,2,5,2019,5,5,329800,7
1057391,충북,휴양콘도 운영업,충북,70s,2,5,2019,10,7,557800,7
1057392,충북,휴양콘도 운영업,충북,70s,2,5,2019,12,3,247800,3


In [ ]:
# 인코딩
# Object 문자로 된 컬럼은 LabelEncoder로 인덱스 도출 (범주형 데이터를 dummies화 시켜서 one-hot 인코딩 사용하는 것과 비슷한 기능)
from sklearn.preprocessing import LabelEncoder

dtypes = data.dtypes
encoders = {}

for column in data.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(data[column])
        encoders[column] = encoder
        
data_num = data.copy()
for column in encoders.keys():
    encoder = encoders[column]
    data_num[column] = encoder.transform(data[column])
    
data_num

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,0,0,0,1,1,1,2019,1,4,311200,4
1,0,0,0,1,1,1,2019,2,3,605000,3
2,0,0,0,1,1,1,2019,6,3,139000,3
3,0,0,0,1,1,1,2019,8,3,27500,3
4,0,0,0,1,1,1,2019,9,3,395500,3
...,...,...,...,...,...,...,...,...,...,...,...
1057389,16,40,16,6,2,5,2019,3,3,148000,4
1057390,16,40,16,6,2,5,2019,5,5,329800,7
1057391,16,40,16,6,2,5,2019,10,7,557800,7
1057392,16,40,16,6,2,5,2019,12,3,247800,3


# Feature Engineering & Initial Modeling

## Data split

In [ ]:
data_num.columns

Index(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD',
       'FLC', 'year', 'month', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

In [ ]:
X = data_num.drop(columns=['CSTMR_CNT', 'AMT', 'CNT'])
Y = np.log1p(data_num['AMT'])

In [ ]:
y_train

293598    12.633629
225069    17.098805
841233    17.800555
325019    13.545015
651593    13.347507
            ...    
409880    11.674628
625347    12.784373
956303    16.812980
485954    15.543620
281734    20.095282
Name: AMT, Length: 518122, dtype: float64

In [ ]:
x_train_all, x_test, y_train_all, y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True)
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size = 0.3, shuffle = True)

In [ ]:
x_train.shape

(482630, 8)

## Modeling

In [ ]:
# Random Forest Model

from sklearn.ensemble import RandomForestRegressor

model_rf1 = RandomForestRegressor(n_estimators=1500,  max_features=8, random_state=0, n_jobs=-1)
model_rf1.fit(x_train, y_train)

In [ ]:
# Verification

from sklearn.metrics import mean_squared_error
pred_rf = model_rf1.predict(x_test)
mse = mean_squared_error(y_test, pred_rf)
print('mse=', mse)
print('rmse=', mse**0.5)

In [ ]:
# Deep Learning Model
# Activation - Relu

from tensorflow.keras import layers

model2 = tf.keras.models.Sequential([
layers.Input(8),

layers.Dense(512, activation='sigmoid'),
layers.Dense(512, activation='sigmoid'),
layers.Dropout(0.3),
    
layers.Dense(256, activation='sigmoid'),
layers.Dense(256, activation='sigmoid'),
layers.Dropout(0.3),
    
layers.Dense(128, activation='sigmoid'),
layers.Dense(128, activation='sigmoid'),
layers.Dropout(0.3),    

layers.Dense(64, activation='sigmoid'),
layers.Dense(64, activation='sigmoid'),
layers.Dropout(0.3),
    
layers.Dense(1) ])

model2.compile(optimizers.Adam(lr=0.01), loss='mean_squared_logarithmic_error',
                 metrics=['mean_squared_logarithmic_error'])

hist = model2.fit(x_train, y_train, epochs=20, batch_size=256,
                    validation_data=(x_val, y_val),callbacks=[early_stop])

Epoch 1/20
2024/2024 [==============================] - 24s 12ms/step - loss: 0.0518 - mean_squared_logarithmic_error: 0.0518 - val_loss: 0.0262 - val_mean_squared_logarithmic_error: 0.0262
Epoch 2/20
2024/2024 [==============================] - 24s 12ms/step - loss: 0.0356 - mean_squared_logarithmic_error: 0.0356 - val_loss: 0.0262 - val_mean_squared_logarithmic_error: 0.0262
Epoch 3/20
2024/2024 [==============================] - 24s 12ms/step - loss: 0.0320 - mean_squared_logarithmic_error: 0.0320 - val_loss: 0.0262 - val_mean_squared_logarithmic_error: 0.0262
Epoch 4/20
2024/2024 [==============================] - 24s 12ms/step - loss: 0.0286 - mean_squared_logarithmic_error: 0.0286 - val_loss: 0.0263 - val_mean_squared_logarithmic_error: 0.0263
Epoch 5/20
2024/2024 [==============================] - 24s 12ms/step - loss: 0.0268 - mean_squared_logarithmic_error: 0.0268 - val_loss: 0.0264 - val_mean_squared_logarithmic_error: 0.0264
Epoch 6/20
2024/2024 [============================

In [19]:
# Deep Learning Model
# add Early Stop , change Dropout value

from tensorflow.keras import layers

model_st = tf.keras.models.Sequential([
layers.Input(8),
layers.Dense(256, activation='relu'),
layers.Dropout(0.3),
layers.Dense(128, activation='relu'),
layers.Dropout(0.3),
layers.Dense(64, activation='relu'),
layers.Dropout(0.3),
layers.Dense(32, activation='relu'),
layers.Dropout(0.3),
layers.Dense(16, activation='relu'),
layers.Dropout(0.3),
layers.Dense(8, activation='relu'),
layers.Dropout(0.3),
layers.Dense(1) ])

from tensorflow.keras import optimizers

model_st.compile(optimizers.Adam(lr=0.01), loss='mean_squared_logarithmic_error', metrics=['mean_squared_logarithmic_error'])

early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'mean_squared_logarithmic_error', patience = 3, restore_best_weights = True)

hist = model_st.fit(x_train, y_train, epochs=100, batch_size=256,
                    validation_data=(x_val, y_val), callbacks=[early_stop])

In [ ]:
# Verification

model_st.evaluate(x_train, y_train)

16192/16192 [==============================] - 14s 877us/step - loss: 0.0262 - mean_squared_logarithmic_error: 0.0262


[0.02617737650871277, 0.02617737650871277]

## Prediction

In [ ]:
test1 = model_rf1.predict(x_test)
test1 = np.expm1(test1)

In [ ]:
test1

array([3.57356498e+06, 1.05379599e+07, 1.71368431e+05, ...,
       5.08365326e+05, 9.26108661e+08, 3.50663249e+05])

# 제출 파일

In [ ]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = jeju['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = jeju['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = jeju['HOM_SIDO_NM'].unique()
AGEs          = jeju['AGE'].unique()
SEX_CTGO_CDs  = jeju['SEX_CTGO_CD'].unique()
FLCs          = jeju['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month'])

In [ ]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,강원,건강보조식품 소매업,강원,20s,1,1,2020,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2020,7
2,강원,건강보조식품 소매업,강원,20s,1,2,2020,4
3,강원,건강보조식품 소매업,강원,20s,1,2,2020,7
4,강원,건강보조식품 소매업,강원,20s,1,3,2020,4
...,...,...,...,...,...,...,...,...
1658855,충북,정기 항공 운송업,제주,10s,2,3,2020,7
1658856,충북,정기 항공 운송업,제주,10s,2,4,2020,4
1658857,충북,정기 항공 운송업,제주,10s,2,4,2020,7
1658858,충북,정기 항공 운송업,제주,10s,2,5,2020,4


In [ ]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658860 entries, 0 to 1658859
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   CARD_SIDO_NM  1658860 non-null  object
 1   STD_CLSS_NM   1658860 non-null  object
 2   HOM_SIDO_NM   1658860 non-null  object
 3   AGE           1658860 non-null  object
 4   SEX_CTGO_CD   1658860 non-null  object
 5   FLC           1658860 non-null  object
 6   year          1658860 non-null  object
 7   month         1658860 non-null  object
dtypes: object(8)
memory usage: 101.2+ MB


In [ ]:
# 인코딩
# Object 문자로 된 컬럼은 LabelEncoder로 인덱스 도출 (범주형 데이터를 dummies화 시켜서 one-hot 인코딩 사용하는 것과 비슷한 기능)
from sklearn.preprocessing import LabelEncoder

dtypes = temp.dtypes
encoders = {}
obj_col = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE']

for column in obj_col:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(temp[column])
        encoders[column] = encoder
        
temp_num = temp.copy()

for column in encoders.keys():
    encoder = encoders[column]
    temp_num[column] = encoder.transform(temp[column])
    
temp_num

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,0,0,1,1,1,2020,4
1,0,0,0,1,1,1,2020,7
2,0,0,0,1,1,2,2020,4
3,0,0,0,1,1,2,2020,7
4,0,0,0,1,1,3,2020,4
...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7
1658856,16,30,14,0,2,4,2020,4
1658857,16,30,14,0,2,4,2020,7
1658858,16,30,14,0,2,5,2020,4


In [ ]:
temp_num = temp_num.astype('int32')
temp_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658860 entries, 0 to 1658859
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype
---  ------        --------------    -----
 0   CARD_SIDO_NM  1658860 non-null  int32
 1   STD_CLSS_NM   1658860 non-null  int32
 2   HOM_SIDO_NM   1658860 non-null  int32
 3   AGE           1658860 non-null  int32
 4   SEX_CTGO_CD   1658860 non-null  int32
 5   FLC           1658860 non-null  int32
 6   year          1658860 non-null  int32
 7   month         1658860 non-null  int32
dtypes: int32(8)
memory usage: 50.6 MB


### Model Predict

In [ ]:
# DeepLearning Model

pred4 = model_st.predict(temp_num)
pred4 = np.round(pred4, 0)
pred4

array([[14.],
       [14.],
       [14.],
       ...,
       [14.],
       [14.],
       [14.]], dtype=float32)

In [ ]:
# Random Forest Model

pred_rf = model_rf1.predict(temp_num)
pred_rf = np.expm1(pred_rf)
pred_rf = np.round(pred_rf, 0)
pred_rf

array([388923., 209429., 168094., ..., 123677., 128984., 123677.])

### Input prediction value to DF

In [18]:
temp_list = []

# # Deeplearning

# for i in range(len(pred4)):
#     temp_list.append(pred4[i][0])
# temp_list

# Random Forest

for i in range(len(pred_rf)):
    temp_list.append(pred_rf[i])
temp_list

In [ ]:
temp_series = pd.Series(temp_list)
temp_series

0          388923.0
1          209429.0
2          168094.0
3          156752.0
4          234400.0
             ...   
1658855    123677.0
1658856    128984.0
1658857    123677.0
1658858    128984.0
1658859    123677.0
Length: 1658860, dtype: float64

In [ ]:
new = pd.concat([temp_num, temp_series], axis=1)
new.rename(columns= {0 : "AMT"}, inplace = True)

new

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT
0,0,0,0,1,1,1,2020,4,388923.0
1,0,0,0,1,1,1,2020,7,209429.0
2,0,0,0,1,1,2,2020,4,168094.0
3,0,0,0,1,1,2,2020,7,156752.0
4,0,0,0,1,1,3,2020,4,234400.0
...,...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7,123677.0
1658856,16,30,14,0,2,4,2020,4,128984.0
1658857,16,30,14,0,2,4,2020,7,123677.0
1658858,16,30,14,0,2,5,2020,4,128984.0


In [ ]:
new['REG_YYMM'] = new['year']*100 + new['month']
new = new[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
new = new.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
new['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(new['CARD_SIDO_NM'])
new['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(new['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('./jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(new, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission_SS.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,5.194644e+08
1,202004,강원,골프장 운영업,6.093961e+09
2,202004,강원,과실 및 채소 소매업,1.776007e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.103811e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.017853e+08
